# Imputing missing values

We work with the data set how it is present now and apply a common machine learning methods to compute the imputations for missing values: 

The **weighted k nearest neighbour (w-kNN)** algorithm, which imputes missing values with weights equal to the inverse Euclidean distance. 

**Assumptions:** we believe the values missing lie in between the boundaries of the highest and lowest value present in the data set. This might work well for most, but it could also be that the missing values are outliers.

### A note on `countries`

We want to **remind** ourselves that our list `groupings` does not only contain countries, it also contains groups of countries like continents, economic zones, etc. and islands which belong to certain countries, but could somehow be very different as e.g. territories. We save all these entries in a new list `countries`. 

Furthermore, some sub-indicators are of ordinal type, i.e. they are defined as 'number of countries which...'. For all countries, we have here either a 1 or a 0 for 'yes' or 'no', respectively. For all other groupings, we have there most likely a number larger than 1. This could be tricky for our imputations later, because these are based on the similarity of two groupings and these similarities could be strong between, say, France and the World Trade Organisation (WTO). If the WTO had a missing value for an ordinal sub-indicator, the imputation would most likely be very similar to the one of France, so 1 or 0. But this is unrealistic, and all other countries being member of the WTO and behave almost as similar as France, would make the imputation just closer to 1 and not larger than 1.

Therefore, we focus on `countries` only from hereon. 

In [24]:
import numpy as np
import pandas as pd
import math
import os
import pickle
import copy
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
# loading original and standardised data set
dict_all = pickle.load(open('utils/data/dict_all_wb.pkl', 'rb'))
dict_all_std = pickle.load(open('utils/data/dict_all_wb_std.pkl', 'rb'))

In [3]:
# check
print('Original values: ')
print(dict_all['Belgium'].loc['EN.CLC.MDAT.ZS'])

print('--------')

print('Standardised values: ')
print(dict_all_std['Belgium'].loc['EN.CLC.MDAT.ZS'])

Original values: 
TimePeriod
2000         NaN
2001         NaN
2002         NaN
2003         NaN
2004         NaN
2005         NaN
2006         NaN
2007         NaN
2008         NaN
2009    0.001692
2010         NaN
2011         NaN
2012         NaN
2013         NaN
2014         NaN
2015         NaN
2016         NaN
2017         NaN
2018         NaN
2019         NaN
Name: EN.CLC.MDAT.ZS, dtype: float64
--------
Standardised values: 
TimePeriod
2000    NaN
2001    NaN
2002    NaN
2003    NaN
2004    NaN
2005    NaN
2006    NaN
2007    NaN
2008    NaN
2009    0.0
2010    NaN
2011    NaN
2012    NaN
2013    NaN
2014    NaN
2015    NaN
2016    NaN
2017    NaN
2018    NaN
2019    NaN
Name: EN.CLC.MDAT.ZS, dtype: float64


Let's calculate the total number of values we have:

In [ ]:
# number of values
s = 0
for country in dict_all_std.keys():
    s += np.sum(dict_all_std[country].count())

print(s)

Now, we open the `csv` file in a GUI and delete the groupings which are *not* countries or part of countries. We call these non-country groupings and examples are North America, Western Asia, Least Developed Countries (LDC), Land Locked Developing Countries (LLDC), Small Island Developing States (SIDS).

In [5]:
# read amended csv file
c = pd.read_csv('utils/countries_wb.csv', dtype=str, delimiter=';', header=None)
countries = list(c[0])

# check
countries

['Afghanistan',
 'Albania',
 'Algeria',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas, The',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo, Dem. Rep.',
 'Congo, Rep.',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt, Arab Rep.',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia, The',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Greenland',
 'Grenada',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Haiti',
 'Honduras',
 '

# 1) weighted k nearest neighbour (w-kNN)

The w-kNN algorithm is straightforward: we calculate how similar countries are in each given year $y$ with the standardised Euclidean distance $E_y$ and take the inverse of the absolute $|E_y|$ as the weight to impute missing values in any given country $c_i$ in each given year $y$ for each given sub-indicator $j$.

A good start to understand this algorithm is to understand high-dimensional space: https://youtu.be/wvsE8jm1GzE

### Euclidean distance
The Euclidean distance $e_y$ for year $y$ for any given pair of countries $(c_i, c_k)$ for any given sub-indicator $j$ is calculated by:
$$ e_y(c_{i}, c_{k}) = \lVert c_{i}, c_{k} \rVert_2 = \sqrt{ \sum_{j=1}^J(c_{ij} - c_{kj})^2} $$

We calculate the squared distances between any given pair of countries $(c_i, c_k)$, but do not consider the country $k+1$ which has the largest distance $e_y$ to country $i$. We do so for any given sub-indicator $j$ and take the square root of it. $c_{ij}$ is the sub-indicator $j$ of country $i$, and $i \neq k$. Thus, any unique pair of countries $i$ and $k$, $i \neq k$, has **one** Euclidean distance $e_y$ for year $y$ only.

Afterwards, we normalise this with respect to the country $k+1$ which has the largest distance $e_y$ to country $i$ by the following equation:

$$ E_y(c_{i}, c_{k}) = \frac{e_y(c_{i}, c_{k})}{e_y(c_{i}, c_{k+1})} $$

This can be seen as equivalent to the well-known normalisation equation:

$$
x_n = \frac{x - x_{min}}{x_{max}-x_{min}}
$$

since $x_{min}$ is always 0, because the distance "between" the same country is 0.

### Imputations
We want that our imputations $x^{j}_{i,y}$ for missing sub-indicator $j$ in country $i$ in year $y$ are similar to sub-indicators $j$ of countries $k$ which have a **small** Euclidean distance $E_y$ and dissimilar to sub-indicators $j$ of countries $k$ which have a **large** Euclidean distance $E_y$. Consequently, the imputations $x^{j}_{i,y}$ are the *weighted* averages where the *weights* are equal to the inverse standardised Euclidean distance $\frac{1}{|E_y(c_{i}, c_{k})|}$.

First, we compute $E_y$ for all available pairs of sub-indicators $j$ amongst two countries $i$ and $k$. Since countries have different amounts of available data points, we average by multiplying the sum by $1/J$, where $J$ is the total number of sub-indicators taken into account here. Note, this does not necessarily be 375, because we have missing values for many sub-indicators. Second, we sum over $k$ to add together all weighted $x^j_{k,y}$ of each unique pair of countries $i$ and $k$ and compute its average by dividing by $K$.

$$ x^{j}_{i,y} = \frac{1}{K} \sum_k \frac{1}{|E_y(c_{i}, c_{k})|} \cdot x^j_{k,y} $$

**Assumptions:** we calculate how similar countries are according to their values for *all* sub-indicators in a given year. We assume that the specific sub-indicators which do not have values in this given year are exactly as similar as the ones which we can calculate a distance for.

Let's have a final check before we start our w-kNN algorithm to compute all missing values. We have here negative values which might sound confusing, but bear in mind that we have standardised the data before, i.e. the data distribution has mean 0 and standard deviation 1.

In [6]:
dict_all_std['Iraq'].head()

TimePeriod,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
EG.CFT.ACCS.ZS,-2.310097,-1.768136,-1.312596,-0.89221,-0.550922,-0.250646,0.007151,0.207823,0.389453,0.515423,0.641392,0.732207,0.799586,0.877218,0.925555,0.976822,1.011976,NaN,NaN,NaN
EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,-1.396716,-0.861510,-1.090495,-0.853583,-0.573230,-0.973504,0.482417,0.568019,1.020090,1.161159,1.250978,1.266375,NaN,NaN
EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,-1.375457,-0.607305,-0.969047,-0.808887,-0.613854,-1.331333,0.631280,0.452946,0.973669,1.139078,1.245289,1.263620,NaN,NaN
EG.ELC.ACCS.UR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,-1.016448,-1.791503,-1.325617,-0.804348,-0.228087,0.931728,-0.391218,0.898583,0.931728,0.931728,0.931728,0.931728,NaN,NaN
FX.OWN.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.744066,NaN,NaN,-0.669492,NaN,NaN,1.413558,NaN,NaN


In [ ]:
# CHECKPOINT
dict_e = pickle.load(open('utils/data/distances_unstd.pkl', 'rb'))
dict_E = pickle.load(open('utils/data/distances_std.pkl', 'rb'))

In [ ]:
# check
print('unstandardised distance e:', dict_e['2000', 'Afghanistan', 'Colombia'])
print('standardised distance E:  ', dict_E['2000', 'Afghanistan', 'Colombia'])

### Calculating the Euclidean distance

We can calculate the standardised distance $E_y(c_{i}, c_{k})$ after having prepared everything. We do this for each unique pair of two *countries* in each year. In other words, we do not want to calculate $E_y(c_{i}, c_{k})$ for $i = k$ and $E_y(c_{i}, c_{k}) = E_y(c_{k}, c_{i})$.

The python package <code>itertools</code> can help us generating the unique pairs of countries.

In [7]:
# create list out of all unique combinations
countrycombinations = list(itertools.combinations(countries, 2))
countrycombinations

[('Afghanistan', 'Albania'),
 ('Afghanistan', 'Algeria'),
 ('Afghanistan', 'Angola'),
 ('Afghanistan', 'Antigua and Barbuda'),
 ('Afghanistan', 'Argentina'),
 ('Afghanistan', 'Armenia'),
 ('Afghanistan', 'Australia'),
 ('Afghanistan', 'Austria'),
 ('Afghanistan', 'Azerbaijan'),
 ('Afghanistan', 'Bahamas, The'),
 ('Afghanistan', 'Bahrain'),
 ('Afghanistan', 'Bangladesh'),
 ('Afghanistan', 'Barbados'),
 ('Afghanistan', 'Belarus'),
 ('Afghanistan', 'Belgium'),
 ('Afghanistan', 'Belize'),
 ('Afghanistan', 'Benin'),
 ('Afghanistan', 'Bhutan'),
 ('Afghanistan', 'Bolivia'),
 ('Afghanistan', 'Bosnia and Herzegovina'),
 ('Afghanistan', 'Botswana'),
 ('Afghanistan', 'Brazil'),
 ('Afghanistan', 'Brunei Darussalam'),
 ('Afghanistan', 'Bulgaria'),
 ('Afghanistan', 'Burkina Faso'),
 ('Afghanistan', 'Burundi'),
 ('Afghanistan', 'Cambodia'),
 ('Afghanistan', 'Cameroon'),
 ('Afghanistan', 'Canada'),
 ('Afghanistan', 'Central African Republic'),
 ('Afghanistan', 'Chad'),
 ('Afghanistan', 'Chile'),
 ('Af

In [8]:
# check
countrycombinations[0][0]

'Afghanistan'

Here, we calculate the standardised distance $E_y(c_{i}, c_{k})$ for each unique pair of two countries in each year.

In [9]:
from scipy.spatial import distance

First, we compute the (not standardised) distances $e_y$ and insert them into a new dictionary `dict_e`.

While exploring the data, we see that nearly no data are available for the years `1990` to `1999`. Consequently, imputations in those years will be based on very weak foundations and we do not consider these years for now. For our similarity investigations later, it does not matter much how many data points we have totally available per country, it is more important that all countries have the same amount of data points. For now, we also omit data for the year `2019`, because it seems not all countries have reported their data yet. Hence, there aren't too many data points available neither.

We set the `period` of years we want to consider in our computations for $e_y$.

In [10]:
period = ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']

In [11]:
# call seriescodes again
info = pd.read_csv('utils/wb_info.csv', dtype=str)
#seriescodes = list(info['Series Code'])
seriescodes = list(dict_all['Germany'].index)
len(seriescodes)

378

In [12]:
# checking if '8.10' became 8.1 (8 is only SDG with a target number 10 in the current version of data)
info[info['SDG']=='13']

,Series Code,Topic,Indicator Name,SDG,target,direction
298,VC_DSR_MTMP,NaN,Number of deaths and missing persons attribute...,13,13.1,-1
299,VC_DSR_DAFF,NaN,Number of directly affected persons attributed...,13,13.1,-1
300,SG_DSR_SILS,NaN,Proportion of local governments that adopt and...,13,13.1,1
301,EN.CLC.DRSK.XQ,Environment: Land use,Disaster risk reduction progress score (1-5 sc...,13,13.2,-1
302,EN.CLC.MDAT.ZS,Environment: Land use,"Droughts, floods, extreme temperatures (% of p...",13,13.1,-1


In [13]:
# check
dict_all_std['Iraq'].shape

(378, 20)

### Comparing Euclidean distances computed with vectors of different dimensions

Simply computing the Euclidean distance between all the countries we have won't give us the results we like, because each of the pairs of countries we calculate the Euclidean distance for has different many measurements to take into account.

We consider this by counting how many measurements `j` we have for each pair of countries and setting the weight `w` as the invesre of the number of measurements.

In [14]:
# ~2 hours computing time
# no need to run every time again, just see CHECKPOINT above and load pickle file

dict_e = {}    

for year in period:
    
    for countrycombination in countrycombinations:
        
        country0_e = []    # create two empty lists for the two groupings we consider at the moment
        country1_e = []    # these lists contain series codes with data available in both groupings
        j = 0    # counter
        
        for seriescode in seriescodes:
            # we can only consider sub-indicators with data available in both groupings
            if pd.isna(dict_all_std[countrycombination[0]].loc[seriescode, year]) is False and pd.isna(dict_all_std[countrycombination[1]].loc[seriescode, year]) is False:
                country0_e.append(dict_all_std[countrycombination[0]].loc[seriescode, year])
                country1_e.append(dict_all_std[countrycombination[1]].loc[seriescode, year])
                
                j += 1
        
        #print('number of data points available: ', j)    # check
        if j > 0:
            e = distance.euclidean(country0_e, country1_e, w=1/j)
        else:
            e = np.nan    # make NaN
            
        #print('e in {} between {} and {}:'.format(year, countrycombination[0], countrycombination[1]), e)
        
        dict_e[year, countrycombination[1], countrycombination[0]] = e
        dict_e[year, countrycombination[0], countrycombination[1]] = dict_e[year, countrycombination[1], countrycombination[0]]


In [15]:
# better save this precious data
f = open('utils/data/distances_unstd.pkl', 'wb')
pickle.dump(dict_e, f)
f.close()

In [27]:
dict_e = pickle.load(open('utils/data/distances_unstd.pkl', 'rb'))

In [28]:
# check
print(dict_e['2011', 'Switzerland', 'Iraq'])

0.6999948906503289


Normalise these distances $e_y$ and save them in `dict_E`:

In [29]:
dict_E = {}

for year in period:
    
    max_e = 0    # maximum value per year
    min_e = 0
    dict_e_year = {}    # auxiliary dictionary with all distances per year
    
    for k in dict_e.keys():
        if year in k:
            dict_e_year[k] = dict_e[k]            

    max_e = np.nanmax(list(dict_e_year.values()))

    #print('------------')
    #print('max_e in {}'.format(year), max_e)
    #print('------------')
       
    for k in dict_e_year.keys():
        #print(k)
        if np.isnan(dict_e_year[k]) == False:
            #print('unstandardised distance e:', dict_e_year[k])
            dict_E[k] = dict_e_year[k] / max_e    # standardise distance
            
        else:
            dict_E[k] = np.nan    # keep as NaN
        
        #print('standardised distance E:', dict_E[k])
        #print('------------')

In [30]:
# check
print('unstandardised distance e:', dict_e['2016', 'Afghanistan', 'Colombia'])
print('standardised distance E:  ', dict_E['2016', 'Afghanistan', 'Colombia'])

unstandardised distance e: 1.0487816277247035
standardised distance E:   0.48719020497650356


In [31]:
# check (both should be False)
print(0 in dict_e.values())
print(0 in dict_E.values())

False
False


In [32]:
# check 
min_value = 0.1

for key, value in dict_E.items():
    if 0 < value < min_value:
        min_value = value
        print('smallest:', key, value)

smallest: ('2003', 'Tuvalu', 'South Sudan') 0.09654472205133677


In [33]:
# better save this precious data
f = open('utils/data/distances_std.pkl', 'wb')
pickle.dump(dict_E, f)
f.close()

### Imputations for countries

Now, we impute the missing values according to the equation we previously derived:

$$ x^{j'}_{i,y} = \frac{1}{K} \sum_k \frac{1}{|E_y(c_{i}, c_{k})|} \cdot x^j_{k,y} $$

To recap, our imputations $x^{j'}_{i,y}$ for missing sub-indicator $j'$ in country $i$ in year $y$ should be similar to sub-indicators $j$ of country $k$, according to the inverse standardised Euclidean distance $\frac{1}{|E_y(c_{i}, c_{k})|}$ between $i$ and $k$. 

As aforementioned and shown in the equation of $E_y$, $E_y$ is dependent on the number of pairs we have in both countries data for. Our `dict_E` has already entries for $E$ normalised according to this number of available pairs of data. We multiply our weight for each imputation, i.e. the inverse standardised Euclidean distance $\frac{1}{|E_y(c_{i}, c_{k})|}$ between $i$ and $k$, by the value $x^j_{k,y}$ of the other country $k$ in year $y$ for sub-indicator $j$. We sum over $k$ to add together all weighted $x^j_{k,y}$ of each unique pair of countries $i$ and $k$ and compute its average by dividing by $K$. $K$ is the number of countries which have values available for the sub-indicator $x^{j'}_{k,y}$ to be computed. 

We also know that some indicators are **binary**, i.e. 1 for 'yes' and 0 for 'no', and ordinal for groupings of countries. These indicators start usually with 'number of countries which...'. The imputations for these must be handled differently: we round the imputed value to an integer.

In [ ]:
# in UN data set exist binary indicators
binary = ['1.5.3', '5.1.1', '5.6.2', '5.a.2', '5.c.1', '8.b.1', '10.7.2', '11.b.1', '12.1.1', '12.7.1', '13.1.2', '13.2.1', '13.3.1', '13.3.2', '13.b.1', '14.c.1', '15.6.1', '15.8.1', '16.10.2', '17.5.1', '17.14.1', '17.16.1', '17.18.2', '17.18.3', '17.19.2']

In [ ]:
info_binary = info.loc[info['Indicator'].isin(binary)]
binary_seriescodes = list(info_binary['SeriesCode'])

binary_seriescodes

In [34]:
# ~30 minutes

dict_all_i = {}

for country in countries:
    
    dict_all_i[country] = pd.DataFrame(index=seriescodes, columns=period)
    
    not_countries = [c for c in countries if c != country]
    
    for seriescode in seriescodes:
        for year in period:            
            if pd.isna(dict_all_std[country].loc[seriescode, year]) is True:
                K = 0
                all_k = []
                
                for not_country in not_countries: 
                    if pd.isna(dict_all_std[not_country].loc[seriescode, year]) is False and pd.isna(dict_E[(year, country, not_country)]) is False: # and dict_E[(year, country, not_country)]!=0:    # not_country can also have NaN -> exclude those
                        K += 1
                        # print('value:', dict_all_std[not_country].loc[seriescode, year])
                        # print('distance:', dict_E[(year, country, not_country)])
                        k = (dict_all_std[not_country].loc[seriescode, year]) / (dict_E[(year, country, not_country)])
                        # print('k =', k)
                        all_k.append(k)
                        
                sum_k = np.sum(all_k)
                    
                #print('K =', K)
                    
                if K > 0:
                    # print('sum k =', sum_k)
                    
                    """
                    if seriescode in binary_seriescodes:
                        dict_all_i[country].loc[seriescode, year] = np.around(sum_k / K)    # round to have binary
                    else:
                        dict_all_i[country].loc[seriescode, year] = sum_k / K
                    """
                    dict_all_i[country].loc[seriescode, year] = sum_k / K
                            
                else:
                    dict_all_i[country].loc[seriescode, year] = np.nan    # only impute when data of other countries is available, 0 cannot be imputed because time-series are non-stationary
                
                #print('Imputation for {} in {} in {}'.format(seriescode, country, year), dict_all_i[country].loc[seriescode, year])        
                
            else:
                dict_all_i[country].loc[seriescode, year] = dict_all_std[country].loc[seriescode, year]

We need to delete all keys which are not part of the list of countries.

In [35]:
# list of keys to delete
delete_keys = []

for key in dict_all_i.keys():
    if key not in countries:
        delete_keys.append(key)
        
# delete
for dk in delete_keys:
    dict_all_i.pop(dk, None)

In [36]:
# check 
max_values = []

for c in dict_all_i.keys():
    max_values.append(dict_all_i[c].max().max())
    
max(max_values)

4.358898943540675

In [37]:
# check
print('NaN here', dict_all_std['Afghanistan'].loc['SE.SEC.UNER.LO.ZS', '2016'])
print('Imputed value', dict_all_i['Afghanistan'].loc['SE.SEC.UNER.LO.ZS', '2016'])

NaN here nan
Imputed value -0.39159814745710453


In [38]:
# check
dict_all_std['Afghanistan'].loc['SE.SEC.UNER.LO.ZS']

TimePeriod
2000   NaN
2001   NaN
2002   NaN
2003   NaN
2004   NaN
2005   NaN
2006   NaN
2007   NaN
2008   NaN
2009   NaN
2010   NaN
2011   NaN
2012   NaN
2013   NaN
2014   NaN
2015   NaN
2016   NaN
2017   NaN
2018   NaN
2019   NaN
Name: SE.SEC.UNER.LO.ZS, dtype: float64

In [39]:
# check
dict_all_i['Afghanistan'].loc['SE.SEC.UNER.LO.ZS']

2000      1.36042
2001      1.25903
2002     0.528416
2003     0.333321
2004     -0.13331
2005    -0.228945
2006    -0.120507
2007   -0.0531405
2008    0.0209203
2009     0.260843
2010    0.0562477
2011   -0.0762566
2012    -0.225768
2013    -0.341088
2014    -0.458794
2015     -0.87171
2016    -0.391598
Name: SE.SEC.UNER.LO.ZS, dtype: object

We want to save the imputations to have another checkpoint here.

In [40]:
# as csv files
if not os.path.exists('csv_imputed'):
    os.mkdir('csv_imputed')

for c in countries:
    dict_all_i[c].to_csv(r'csv_imputed/{}_wb.csv'.format(c))
    
# as pkl files
imp = open('utils/data/dict_all_i_wb.pkl', 'wb')
pickle.dump(dict_all_i, imp)
imp.close()

In [ ]:
# CHECKPOINT
dict_all_i = pickle.load(open('utils/data/dict_all_i_wb.pkl', 'rb'))

Let's calculate the total number of values:

In [41]:
# number of values
s_imp = 0
for country in dict_all_i.keys():
    s_imp += np.sum(dict_all_i[country].count())

print(s_imp)

1005219


# Averaging and concatenating data to higher levels

### *(UN data set only)* Averaging and concatenating series codes data to indicator-level

1. We can average all series codes, i.e. sub-indicators, belonging to one indicator to this indicator.
2. We can see the series codes as multiple samples of the same indicator and concatenate the series codes into indicators. Consequently, we have more than one measurement per time point for any indicators having more than one series code.

In [ ]:
indicators = list(info.Indicator)

dict_indicators = {}

for indicator in indicators:
    i = info['SeriesCode'].where(info['Indicator'] == indicator)

    dict_indicators[indicator] = [s for s in i if str(s) != 'nan']

In [ ]:
# check
np.isnan(dict_all_i['Germany'].loc['SI_POV_DAY1', '2001'])

In [ ]:
#indicators_values = {}
#indicators_values_std = {}
indicators_values_i = {}

for country in countries:
    #print(country)
    
    #indicators_values[country] = pd.DataFrame(columns=period, index=indicators)
    #indicators_values_std[country] = pd.DataFrame(columns=period, index=indicators)
    indicators_values_i[country] = pd.DataFrame(columns=period, index=list(dict_indicators.keys()))
    
    for year in period:
        
        for indicator in dict_indicators.keys():
            #list_subindicators_values = []
            #list_subindicators_values_std = []
            list_subindicators_values_i = []
    
            for subindicator in list(dict_indicators[indicator]):
                if np.isnan(dict_all_i[country].loc[subindicator, year]):
                    pass
                else:
                    #list_subindicators_values.append(dict_all[country].loc[subindicator, year])
                    #list_subindicators_values_std.append(dict_all_std[country].loc[subindicator, year])
                    list_subindicators_values_i.append(dict_all_i[country].loc[subindicator, year])
            
            # 1. averaging
            #indicators_values[country].loc[indicator, year] = np.nanmean(list_subindicators_values)
            #indicators_values_std[country].loc[indicator, year] = np.nanmean(list_subindicators_values_std)
            #indicators_values_i[country].loc[indicator, year] = np.nanmean(list_subindicators_values_i)
            
            # 2. concatenating
            array_subindicators_values_i = np.asarray(list_subindicators_values_i)
            indicators_values_i[country].loc[indicator, year] = array_subindicators_values_i

In [ ]:
# check (duplicate indicator labels should be in index)
indicators_values_i['Germany']

In [ ]:
# better save these precious data
#ind_val = open('utils/data/indicators_values.pkl', 'wb')
#ind_val_std = open('utils/data/indicators_values_std.pkl', 'wb')
ind_val_i = open('utils/data/indicators_values_i.pkl', 'wb')
#pickle.dump(indicators_values, ind_val)
#pickle.dump(indicators_values_std, ind_val_std)
pickle.dump(indicators_values_i, ind_val_i)
#ind_val.close()
#ind_val_std.close()
ind_val_i.close()

### *(UN data set)* Averaging and concatenating indicator data to target-level
We must also generate two lists of indicators which are meant to increase and decrease over time.

In [ ]:
# gone though all targets by hand and checked which indicators are meant to increase and which are meant to decrease over time.
increase = ['1.3.1', '1.4.1', '1.4.2', '1.5.3', '1.5.4', '1.a.1', '1.a.2', '1.a.3', '1.b.1', '2.3.1', '2.3.2', '2.4.1', '2.5.1', '2.a.1', '2.a.2', '3.1.2', '3.5.1', '3.7.1', '3.8.1', '3.b.1', '3.b.2', '3.b.3', '3.c.1', '3.d.1', '4.1.1', '4.2.1', '4.2.2', '4.3.1', '4.4.1', '4.6.1', '4.7.1', '4.a.1', '4.b.1', '4.c.1', '5.1.1', '5.5.1', '5.5.2', '5.6.1', '5.6.2', '5.a.1', '5.a.2', '5.b.1', '5.c.1', '6.1.1', '6.2.1', '6.3.1', '6.3.2', '6.4.1', '6.5.1', '6.5.2', '6.6.1', '6.a.1', '6.b.1', '7.1.1', '7.1.2', '7.2.1', '7.3.1', '7.a.1', '7.b.1', '8.1.1', '8.2.1', '8.3.1', '8.5.1', '8.8.2', '8.9.1', '8.9.2', '8.10.1', '8.10.2', '8.a.1', '8.b.1', '9.1.1', '9.1.2', '9.2.1', '9.2.2', '9.3.1', '9.3.2', '9.5.1', '9.5.2', '9.a.1', '9.b.1', '9.c.1', '10.1.1', '10.4.1', '10.5.1', '10.6.1', '10.7.2', '10.a.1', '10.b.1', '11.2.1', '11.3.2', '11.4.1', '11.6.1', '11.7.1', '11.a.1', '11.b.1', '11.b.2', '11.c.1', '12.1.1', '12.4.1', '12.5.1', '12.6.1', '12.7.1', '12.8.1', '12.a.1', '12.b.1', '13.1.2', '13.1.3', '13.2.1', '13.3.1', '13.3.2', '13.a.1', '13.b.1', '14.2.1', '14.3.1', '14.4.1', '14.5.1', '14.6.1', '14.7.1', '14.a.1', '14.b.1', '14.c.1', '15.1.1', '15.1.2', '15.2.1', '15.4.1', '15.4.2', '15.6.1', '15.8.1', '15.9.1', '15.a.1', '15.b.1', '16.1.4', '16.6.2', '16.7.1', '16.7.2', '16.8.1', '16.9.1', '16.10.2', '16.a.1', '17.1.1', '17.1.2', '17.2.1', '17.3.1', '17.3.2', '17.4.1', '17.5.1', '17.6.1', '17.6.2', '17.7.1', '17.8.1', '17.9.1', '17.11.1', '17.13.1', '17.14.1', '17.15.1', '17.16.1', '17.17.1', '17.18.1', '17.18.2', '17.18.3', '17.19.1', '17.19.2']
decrease = ['1.1.1', '1.2.1', '1.2.2', '1.5.1', '1.5.2', '2.1.1', '2.1.2', '2.2.1', '2.2.2', '2.5.2','2.b.1', '2.c.1', '3.1.1', '3.2.1', '3.2.2', '3.3.1', '3.3.2', '3.3.3', '3.3.4', '3.3.5', '3.4.1', '3.4.2', '3.5.2', '3.6.1', '3.7.2', '3.8.2', '3.9.1', '3.9.2', '3.9.3', '3.a.1', '4.5.1', '5.2.1', '5.2.2', '5.3.1', '5.3.2', '5.4.1', '6.4.2', '8.4.1', '8.4.2', '8.5.2', '8.6.1', '8.7.1', '8.8.1', '9.4.1', '10.2.1', '10.3.1', '10.7.1', '10.c.1', '11.1.1', '11.3.1', '11.5.1', '11.5.2', '11.6.2', '11.7.2', '12.2.1', '12.2.2', '12.3.1', '12.4.2', '12.c.1', '13.1.1', '14.1.1', '15.3.1', '15.5.1', '15.7.1', '15.c.1', '16.1.1', '16.1.2', '16.1.3', '16.2.1', '16.2.2', '16.2.3', '16.3.1', '16.3.2', '16.4.1', '16.4.2', '16.5.1', '16.5.2', '16.6.1', '16.10.1', '16.b.1', '17.10.1', '17.12.1']

In [ ]:
# making all time-series "pointing" upwards when they are meant to increase

indicators_values_i_up = {}

for country in countries:
    indicators_values_i_up[country] = pd.DataFrame(index=list(dict_indicators.keys()), columns=period)
    
    for indicator in dict_indicators.keys():
        if indicator in decrease:
            #indicators_values_up[country].loc[indicator] = indicators_values[country].loc[indicator]*(-1)
            #indicators_values_std_up[country].loc[indicator] = indicators_values_std[country].loc[indicator]*(-1)
            indicators_values_i_up[country].loc[indicator] = list(np.multiply(list(indicators_values_i[country].loc[indicator]), -1))
        else:
            #indicators_values_up[country].loc[indicator] = indicators_values[country].loc[indicator]
            #indicators_values_std_up[country].loc[indicator] = indicators_values_std[country].loc[indicator]
            indicators_values_i_up[country].loc[indicator] = indicators_values_i[country].loc[indicator]

In [ ]:
# check
indicators_values_i_up['Germany']

In [ ]:
# better save these precious data
#ind_val = open('utils/data/indicators_values_up.pkl', 'wb')
#ind_val_std = open('utils/data/indicators_values_std_up.pkl', 'wb')
ind_val_i = open('utils/data/indicators_values_i_up.pkl', 'wb')
#pickle.dump(indicators_values_up, ind_val)
#pickle.dump(indicators_values_std_up, ind_val_std)
pickle.dump(indicators_values_i_up, ind_val_i)
#ind_val.close()
#ind_val_std.close()
ind_val_i.close()

Defining dictionaries for targets:

In [ ]:
targets = list(info['Target'].unique())

dict_targets = {}

for target in targets:
    t = info['Indicator'].where(info['Target'] == target)

    dict_targets[target] = [i for i in t if str(i) != 'nan']

In [ ]:
# check
list(indicators_values_i_up['Germany'].loc['1.1.1', '2000'])

Now, we can simply average or concatenate:

In [ ]:
targets_values_i = {}
#targets_values_up = {}
#targets_values_std_up = {}
targets_values_i_up = {}    # for Granger-causality

for country in countries:
    
    #targets_values_up[country] = pd.DataFrame(columns=period, index=targets)
    #targets_values_std_up[country] = pd.DataFrame(columns=period, index=targets)
    targets_values_i[country] = pd.DataFrame(columns=period, index=list(dict_targets.keys()))
    targets_values_i_up[country] = pd.DataFrame(columns=period, index=list(dict_targets.keys()))
    
    for year in period:
        
        for target in list(dict_targets.keys()):
            #list_indicators_values = []
            #list_indicators_values_std = []
            list_indicators_values_i = []
            list_indicators_values_i_up = []
    
            for indicator in list(dict_targets[target]):
                #list_indicators_values.append(indicators_values[country].loc[indicator, year])
                #list_indicators_values_std.append(indicators_values_std[country].loc[indicator, year])
                list_indicators_values_i.extend(indicators_values_i[country].loc[indicator, year])
                list_indicators_values_i_up.extend(indicators_values_i_up[country].loc[indicator, year])
    
            #print(list_indicators_values_i)
            
            # 1. averaging
            #targets_values_up[country].loc[target, year] = np.mean(list_indicators_values)
            #targets_values_std_up[country].loc[target, year] = np.mean(list_indicators_values_std)
            #targets_values_i_up[country].loc[target, year] = np.mean(list_indicators_values_i)
            
            # 2. concatenating
            targets_values_i[country].loc[target, year] = list_indicators_values_i
            targets_values_i_up[country].loc[target, year] = list_indicators_values_i_up

In [ ]:
# check (each goal should have list in cells with values for sub-indicators)
targets_values_i_up['Germany'].loc['1.1']

The first inner parentheses in each cell contains the values from the first indicator; the second inner parentheses in each cell contains the values from the second indicator; etc. Each indicator represents a couple of sub-indicators.

In [ ]:
# better save these precious data
#tar_val = open('utils/data/targets_values_up.pkl', 'wb')
#tar_val_std = open('utils/data/targets_values_std_up.pkl', 'wb')
tar_val_i = open('utils/data/targets_values_i.pkl', 'wb')
tar_val_i_up = open('utils/data/targets_values_i_up.pkl', 'wb')
#pickle.dump(targets_values_up, tar_val)
#pickle.dump(targets_values_std_up, tar_val_std)
pickle.dump(targets_values_i, tar_val_i)
pickle.dump(targets_values_i_up, tar_val_i_up)
#tar_val.close()
#tar_val_std.close()
tar_val_i.close()
tar_val_i_up.close()

### *(UN data set)* Averaging and concatenating target data to goal-level

Defining dictionaries for goals.

In [ ]:
goals = list(info['Goal'].unique())

dict_goals = {}

for goal in goals:
    g = info['Target'].where(info['Goal'] == goal)

    dict_goals[goal] = [t for t in g if str(t) != 'nan']

In [ ]:
goals_values_i = {}
#goals_values_up = {}
#goals_values_std_up = {}
goals_values_i_up = {}    # for Granger-causality

for country in countries:
    
    #goals_values_up[country] = pd.DataFrame(columns=period, index=goals)
    #goals_values_std_up[country] = pd.DataFrame(columns=period, index=goals)
    goals_values_i[country] = pd.DataFrame(columns=period, index=list(dict_goals.keys()))
    goals_values_i_up[country] = pd.DataFrame(columns=period, index=list(dict_goals.keys()))
    
    for year in period:
        
        for goal in goals:
            #list_targets_values = []
            #list_targets_values_std = []
            list_targets_values_i = []
            list_targets_values_i_up = []
    
            for target in list(dict_goals[goal]):
                #list_targets_values.append(targets_values_up[country].loc[target, year])
                #list_targets_values_std.append(targets_values_std_up[country].loc[target, year])
                list_targets_values_i.extend(targets_values_i[country].loc[target, year])
                list_targets_values_i_up.extend(targets_values_i_up[country].loc[target, year])
    
            #print(list_targets_values_i)
            
            # 1. averaging
            #goals_values_up[country].loc[goal, year] = np.mean(list_targets_values)
            #goals_values_std_up[country].loc[goal, year] = np.mean(list_targets_values_std)
            #goals_values_i_up[country].loc[goal, year] = np.mean(list_targets_values_i)
            
            # 2. concatenating
            goals_values_i[country].loc[goal, year] = list_targets_values_i
            goals_values_i_up[country].loc[goal, year] = list_targets_values_i_up

In [ ]:
# check (each goal should have list in cells with values for sub-indicators)
goals_values_i_up['Germany'].loc['13']

In [ ]:
# better save these precious data
#goa_val = open('utils/data/goals_values_up.pkl', 'wb')
#goa_val_std = open('utils/data/goals_values_std_up.pkl', 'wb')
goa_val_i = open('utils/data/goals_values_i.pkl', 'wb')
goa_val_i_up = open('utils/data/goals_values_i_up.pkl', 'wb')
#pickle.dump(goals_values_up, goa_val)
#pickle.dump(goals_values_std_up, goa_val_std)
pickle.dump(goals_values_i, goa_val_i)
pickle.dump(goals_values_i_up, goa_val_i_up)
#goa_val.close()
#goa_val_std.close()
goa_val_i.close()
goa_val_i_up.close()

### *(WorldBank data set)* Concatenating indicator data to target-level
As we have done with the UN data set, we concatenate indicator data to target-level. We jump over the indicators, because sub-indicators are not mapped to specific indicators in the WorldBank data set, only to targets directly.

In [42]:
# check
dict_all_i['Afghanistan']

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
EG.CFT.ACCS.ZS,-1.35195,-1.25444,-1.13359,-0.986638,-0.853422,-0.707846,-0.526563,-0.364506,-0.165369,0.0269017,0.2796,0.506205,0.746543,1.03358,1.28353,1.57331,1.89467
EG.ELC.ACCS.ZS,-1.43368,-1.26573,-1.06848,-1.00274,-0.768292,-1.382,-1.22622,-0.984912,-0.612862,-0.500381,-0.600968,-0.580272,0.445691,0.487458,1.25447,0.540842,1.57957
EG.ELC.ACCS.RU.ZS,-1.31536,-1.02486,-0.867609,-0.773497,-0.542062,-1.31975,-1.21567,-0.979744,-0.665801,-0.503406,-0.591509,-0.6127,0.41336,0.479844,1.25489,0.535538,1.6026
EG.ELC.ACCS.UR.ZS,-0.884945,-1.07784,-0.950471,-0.999114,-0.912868,-1.92669,-1.17449,-0.944866,0.104129,-0.48063,-0.802715,-0.321477,0.755525,0.466322,1.22811,0.436213,1.33029
FX.OWN.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.88423,NaN,NaN,-0.513711,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SP.M15.2024.FE.ZS,1.27116,1.29361,0.411725,0.238042,0.951381,0.284188,0.359242,1.01283,0.160396,0.674941,0.229309,-0.219094,-0.680828,-0.963139,-0.587686,0,-1.4731
SP.M18.2024.FE.ZS,1.18184,1.91945,1.69796,1.67501,0.686052,0.882849,0.275753,0.667441,0.732022,0.311548,-0.089585,1.1875,-0.579714,-1.47886,-1.28511,-0.634419,-1.63677
VC_DSR_MTMP,NaN,NaN,NaN,NaN,NaN,0.184075,-0.213152,0.31731,0.676016,0.178384,0.225871,-0.020905,-0.452117,-0.650738,-0.647256,0.28237,0.267587
VC_DSR_DAFF,NaN,NaN,NaN,NaN,NaN,-0.199006,-0.46193,0.311593,0.228117,0.0655463,0.390954,-0.0269723,-0.364727,-0.493838,-0.534605,-0.0275088,0.355569


We append these temperature series to the data frame of indicators. 

In [43]:
temp = pickle.load(open('utils/data/temp.pkl', 'rb'))

*Hint:* When the data are downloaded, the names of countries do not match exactly. For example 'Republic of Serbia' is the name in the temperature data set, whereas 'Serbia' is the name in the SDG data set. These are aligned manually.

In [44]:
temp['Serbia'] = temp.pop('Republic of Serbia')
temp['Montenegro'] = temp.pop('Republic of Montenegro')
temp['Timor-Leste'] = temp.pop('Timor Leste')
temp['Slovak Republic'] = temp.pop('Slovakia')
temp['Micronesia, Fed. Sts.'] = temp.pop('Federated States of Micronesia')
temp['Yemen, Rep.'] = temp.pop('Yemen')
temp['Syrian Arab Republic'] = temp.pop('Syria')
temp.pop('Swaziland')
temp['Egypt, Arab Rep.'] = temp.pop('Egypt')
temp['Myanmar'] = temp.pop('Myanmar (Burma)')
temp['Congo, Dem. Rep.'] = temp.pop('Congo (Democratic Republic of the)')
temp['Bahamas, The'] = temp.pop('Bahamas')
temp.pop('Northern Mariana Islands')
temp.pop('Marshall Islands')
temp.pop('Monaco')
temp.pop('St. Vincent and the Grenadines')
temp.pop('St. Lucia')
temp.pop('Andorra')
temp.pop('Faroe Islands')
temp.pop('Cape Verde')
temp.pop('Macedonia')
temp['Congo, Rep.'] = temp.pop('Congo (Republic of the)')
temp['Iran, Islamic Rep.'] = temp.pop('Iran')
temp['Brunei Darussalam'] = temp.pop('Brunei')
temp.pop('St. Kitts and Nevis')
temp['Kyrgyz Republic'] = temp.pop('Kyrgyzstan')
temp['Venezuela, RB'] = temp.pop('Venezuela')
temp.pop('New Caledonia')
temp['Lao PDR'] = temp.pop('Laos')
temp['Russian Federation'] = temp.pop('Russia')
temp['Korea, Dem. People\'s Rep.'] = temp.pop('Korea')
temp['Gambia, The'] = temp.pop('Gambia')

In [45]:
dict_all_t = {}

for country in temp.keys():
    temp[country].index = temp[country].index.astype(int).astype(str)
    dict_all_t[country] = dict_all_i[country].append(temp[country])

In [46]:
countries_to_drop = list(set(list(dict_all_t.keys())).difference(list(temp.keys())))

for c in countries_to_drop:
    dict_all_t.pop(c)

In [48]:
#check 
dict_all_t['France']

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2016,1991,1992,1993,1994,1995,1996,1997,1998,1999
EG.CFT.ACCS.ZS,0,0,0,0,0,0,0,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EG.ELC.ACCS.ZS,0,0,0,0,0,0,0,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EG.ELC.ACCS.RU.ZS,0,0,0,0,0,0,0,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EG.ELC.ACCS.UR.ZS,0,0,0,0,0,0,0,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FX.OWN.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SP.M18.2024.FE.ZS,1.0782,1.63736,1.68158,1.45041,0.71333,0.793987,0.278298,0.637913,-0.709815,0.338247,...,-1.2614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VC_DSR_MTMP,NaN,NaN,NaN,NaN,NaN,0.156264,-0.258692,0.360365,0.616697,0.173229,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VC_DSR_DAFF,NaN,NaN,NaN,NaN,NaN,-0.18115,-0.500321,0.340545,0.23151,0.103391,...,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SG_DSR_SILS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.194265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# CHECKPOINT
indicators_values_i_up = pickle.load(open('utils/data/indicators_values_i_up_wb.pkl', 'rb'))

In [49]:
# making all time-series "pointing" upwards when they are meant to increase

indicators_values_i_up = {}

for country in countries:
    indicators_values_i_up[country] = pd.DataFrame(index=list(dict_all_t[country].index), columns=period)
    #dict_all_t[country].drop(columns=['2017', '2018', '2019'], inplace=True)
    dict_all_t[country].drop(columns=['1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999'], inplace=True)
    
    for seriescode in list(dict_all_t['France'].index):
        indicators_values_i_up[country].at[seriescode] = list(np.multiply(list(dict_all_t[country].loc[seriescode]), int(info.loc[info['Series Code'] == seriescode].direction)))

In [50]:
# check
dict_all_t['Brazil'].loc['EN.CLC.MDAT.ZS']

2000    NaN
2001    NaN
2002    NaN
2003    NaN
2004    NaN
2005    NaN
2006    NaN
2007    NaN
2008    NaN
2009      0
2010    NaN
2011    NaN
2012    NaN
2013    NaN
2014    NaN
2015    NaN
2016    NaN
Name: EN.CLC.MDAT.ZS, dtype: object

In [51]:
# check
indicators_values_i_up['Brazil'].loc['EN.CLC.MDAT.ZS']

2000    NaN
2001    NaN
2002    NaN
2003    NaN
2004    NaN
2005    NaN
2006    NaN
2007    NaN
2008    NaN
2009     -0
2010    NaN
2011    NaN
2012    NaN
2013    NaN
2014    NaN
2015    NaN
2016    NaN
Name: EN.CLC.MDAT.ZS, dtype: object

In [52]:
# better save these precious data
ind_val_i = open('utils/data/indicators_values_i_up_wb.pkl', 'wb')
pickle.dump(indicators_values_i_up, ind_val_i)
ind_val_i.close()

Defining dictionaries for targets:

In [53]:
targets = list(info['target'].unique())

dict_targets = {}

for target in targets:
    t = info['Series Code'].where(info['target'] == target)

    dict_targets[target] = [i for i in t if str(i) != 'nan']

Target values:

In [54]:
targets_values_i = {}
targets_values_i_up = {}

for country in countries:
    targets_values_i[country] = pd.DataFrame(columns=period, index=list(dict_targets.keys()))
    targets_values_i_up[country] = pd.DataFrame(columns=period, index=list(dict_targets.keys()))
    
    for year in period:
        for target in list(dict_targets.keys()):
            list_indicators_values_i = []
            list_indicators_values_i_up = []
    
            for indicator in list(dict_targets[target]):
                # do not append NaNs
                if np.isnan(dict_all_t[country].loc[indicator, year]) == True:
                    continue
                else:
                    list_indicators_values_i.append(dict_all_t[country].loc[indicator, year])      
                
                if np.isnan(indicators_values_i_up[country].loc[indicator, year]) == True:
                    continue
                else:
                    list_indicators_values_i_up.append(indicators_values_i_up[country].loc[indicator, year])
            
            # 1. averaging
            #targets_values_i[country].loc[target, year] = np.mean(list_indicators_values_i)
            #targets_values_i_up[country].loc[target, year] = np.mean(list_indicators_values_i_up)
            
            # 2. concatenating
            targets_values_i[country].loc[target, year] = list_indicators_values_i
            targets_values_i_up[country].loc[target, year] = list_indicators_values_i_up

In [55]:
# check (each target should have list in cells with values for indicators)
targets_values_i_up['Germany'].loc['13.1', '2009']

[-0.16424547179990934, -0.09065964907067506, -0.0]

In [56]:
# better save these precious data
tar_val_i = open('utils/data/targets_values_i_wb.pkl', 'wb')
tar_val_i_up = open('utils/data/targets_values_i_up_wb.pkl', 'wb')
pickle.dump(targets_values_i, tar_val_i)
pickle.dump(targets_values_i_up, tar_val_i_up)
tar_val_i.close()
tar_val_i_up.close()

### *(WorldBank data set)* Concatenating target data to goal-level

Defining dictionaries for goals.

In [57]:
goals = list(info['SDG'].unique())

dict_goals = {}

for goal in goals:
    g = info['target'].where(info['SDG'] == goal)

    dict_goals[goal] = [t for t in g if str(t) != 'nan']

Concatenating:

In [58]:
goals_values_i = {}
goals_values_i_up = {}

for country in countries:
    goals_values_i[country] = pd.DataFrame(columns=period, index=list(dict_goals.keys()))
    goals_values_i_up[country] = pd.DataFrame(columns=period, index=list(dict_goals.keys()))
    
    for year in period:
        
        for goal in goals:
            list_targets_values_i = []
            list_targets_values_i_up = []
    
            for target in list(dict_goals[goal]):
                list_targets_values_i.extend(targets_values_i[country].loc[target, year])
                list_targets_values_i_up.extend(targets_values_i_up[country].loc[target, year])
            
            # 1. averaging
            #goals_values_i[country].loc[goal, year] = np.mean(list_targets_values_i)
            #goals_values_i_up[country].loc[goal, year] = np.mean(list_targets_values_i_up)
            
            # 2. concatenating
            goals_values_i[country].loc[goal, year] = list_targets_values_i
            goals_values_i_up[country].loc[goal, year] = list_targets_values_i_up

In [59]:
# check
goals_values_i_up['France'].loc['13']

2000                                                   []
2001                                                   []
2002                                                   []
2003                                                   []
2004                                                   []
2005    [-0.1562643405328236, 0.1811502496594613, -0.1...
2006    [0.25869215083655683, 0.5003208496401187, 0.25...
2007    [-0.360364776061722, -0.34054549098412157, -0....
2008    [-0.6166965107632474, -0.23151041716941603, -0...
2009    [-0.17322896070990132, -0.1033907654316606, -0...
2010    [-0.22070175786866292, -0.4294252507979116, -0...
2011    [0.06728615894131047, 0.014275424796858022, 0....
2012    [0.5103130741032781, 0.35129886684840167, 0.51...
2013    [0.6077034352965636, 0.44152255584713684, 0.60...
2014    [0.5575653853638209, 0.42459161916378546, 0.55...
2015    [-0.21781092259888551, 0.0815850770529942, -0....
2016    [-1.0, 1.0, -0.1942651792035716, -1.0, 1.0, -0...
Name: 13, dtyp

In [60]:
# better save these precious data
goa_val_i = open('utils/data/goals_values_i_wb.pkl', 'wb')
goa_val_i_up = open('utils/data/goals_values_i_up_wb.pkl', 'wb')
pickle.dump(goals_values_i, goa_val_i)
pickle.dump(goals_values_i_up, goa_val_i_up)
goa_val_i.close()
goa_val_i_up.close()